In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import main as m

In [ ]:
model = m.Model()


In [ ]:
def set_if_within_bounds(x, y, value):
    # Set the value of grid[x,y] to value if x and y are within bounds
    if x >= 0 and x < WIDTH and y >= 0 and y < HEIGHT:
        grid[x,y] = value


In [ ]:
# plot the grid
plt.imshow(model.grid, cmap='Set1')
plt.clim(0,10)
plt.show()

In [ ]:
model.loop(1000)

In [ ]:
plt.imshow(model.grid, cmap='Set1')
plt.title("Final grid 2")
plt.show()